## IMPORTS

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils
import multiprocessing as mp
from sklearn.mixture import GaussianMixture
import seaborn as sns

## DEFINIR PATH DE DATOS

In [2]:
IMG_PATH ='data/'

## CARGAR DATOS

In [3]:
data = utils.get_data(IMG_PATH)

## FUNCION DE EJECUCION PARALELA

In [6]:
def thread_function(subset):
    """
    thread_function

    :param subset: subconjunto de datos
    :Realiza GMM sobre los elementos del subconjunto de datos
    """
    for i in range(len(subset)):
        gmm = GaussianMixture(n_components=5, init_params='kmeans',  n_init=2)
        gmm.fit(subset[i][1][['R','G','B']])
        clusters = gmm.predict(subset[i][1][['R','G','B']])

        iter_item = subset[i][1].copy()
        iter_item['cluster'] = clusters
        iter_item.to_csv(f"Resultados/GMM/Objetos_Clusterizados/{subset[i][0]}.csv")

## EJECUCION DE N PROCESOS PARALELOS

In [7]:
number_processes = mp.cpu_count()
data_arr = np.asarray(data)
subsets = np.array_split(data_arr, number_processes)
procesos = []

for index in range(number_processes):
    procesos.append(mp.Process(target=thread_function,  args=(subsets[index],)))

for process in procesos:
    process.start()

for process in procesos:
    process.join()